In [17]:
# 새로 짠 코드

# import
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# gpu 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [18]:

# 정규화에 쓰일 파라미터 정의 (평균, 분산)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST('./data', train=True, download=True,
                               transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True,
                              transform=transform)

# batch_size는 64로 설정하고 데이터 로더를 사용
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)




In [19]:

# 모델정의
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(-1, 784)
        output = self.linear(x)
        return output



In [20]:
# 정의
lr = 0.01
input_dim = 784
output_dim = 10
model = LogisticRegression(input_dim,output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)



In [21]:
# Train
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # forward
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward & 갱신
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 출력
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))



Epoch [1/10], Step [100/938], Loss: 0.5856
Epoch [1/10], Step [200/938], Loss: 0.3794
Epoch [1/10], Step [300/938], Loss: 0.4731
Epoch [1/10], Step [400/938], Loss: 0.3762
Epoch [1/10], Step [500/938], Loss: 0.2879
Epoch [1/10], Step [600/938], Loss: 0.3710
Epoch [1/10], Step [700/938], Loss: 0.3607
Epoch [1/10], Step [800/938], Loss: 0.3182
Epoch [1/10], Step [900/938], Loss: 0.3902
Epoch [2/10], Step [100/938], Loss: 0.1956
Epoch [2/10], Step [200/938], Loss: 0.2843
Epoch [2/10], Step [300/938], Loss: 0.3090
Epoch [2/10], Step [400/938], Loss: 0.2990
Epoch [2/10], Step [500/938], Loss: 0.2333
Epoch [2/10], Step [600/938], Loss: 0.2006
Epoch [2/10], Step [700/938], Loss: 0.4115
Epoch [2/10], Step [800/938], Loss: 0.2633
Epoch [2/10], Step [900/938], Loss: 0.3118
Epoch [3/10], Step [100/938], Loss: 0.5336
Epoch [3/10], Step [200/938], Loss: 0.2954
Epoch [3/10], Step [300/938], Loss: 0.2669
Epoch [3/10], Step [400/938], Loss: 0.3195
Epoch [3/10], Step [500/938], Loss: 0.3621
Epoch [3/10

In [22]:
# Test
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('정확도: {} %'.format(100 * correct / total))

정확도: 92.24 %
